# Pipeline

1. Preprocessing
2. Data Extraction
3. Data Exploration
4. **Model**

This file initializes the model and makes predictions too.

# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [4]:
# Get feature dataframe from pickle file
df = pd.read_pickle('feature_frame.pkl')

In [6]:
df.sample(5)

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,RND_CNT_league,H_RTG_home,A_RTG_home,H_RTG_away,A_RTG_away,EGD,EPTS_PR_home,EPTS_PR_away,RED_home,RED_away
3784,43,42,1,2,2,0.450000,0.100000,0.550000,0.150000,0.700000,...,34,0.000000,-0.179242,0.000000,-0.597475,0.581824,0.50,0.50,0,2
2313,29,26,1,1,0,0.263158,0.315789,0.157895,0.263158,0.300000,...,38,0.193164,-0.021125,0.057949,-0.070416,0.215347,1.50,0.00,0,0
11998,109,116,1,1,0,0.459459,0.243243,0.307692,0.282051,0.230769,...,38,0.496727,-0.119241,0.149018,-0.397471,0.820838,1.75,0.25,0,0
8254,81,86,2,1,1,0.638889,0.250000,0.410256,0.282051,0.447368,...,38,0.028180,0.059041,0.008454,0.196805,-0.141194,1.00,1.00,1,0
5792,40,50,3,0,1,0.588235,0.323529,0.294118,0.352941,0.235294,...,34,-0.041956,0.069282,-0.012587,0.230939,-0.226659,1.25,1.25,0,1


# Extracting target variables and features

In [7]:
# If True, the goal difference for a specific game is seen as the target variable.
# (e.g. -3 for a game outcome of 1:4, or 2 for 3:1) .
#
# If False, we just want to predict the winner.
# 1 = Home team wins, 0 = Draw, 2 = Away team wins
predict_goal_difference = False

if predict_goal_difference:
    y = df['FTHG'] - df['FTAG']
else:
    y = df['FTR']

X = df.iloc[:,5:]    # Remove unnecessary columns (IDs etc.) from features

# Ranked Probability Score (RPS)

In [8]:
def rps(pred, actual_value, r=3):
    '''Returns the ranked probability score for a single given game.
    (see Hubacek paper for formula)
    
    Arguments:
    pred -- predicted results; in vector form (e.g. [0.1, 0.6, 0.3])
    actual_value -- actual result (0, 1 or 2); not in vector form yet
    r -- number of categories (3 for football)
    '''
    value_vec = [0, 0, 0]
    
    # Bring value_vec into 1, 0, 2 order
    if actual_value == 0:
        value_vec[1] = 1
    elif actual_value == 2:
        value_vec[0] == 1
    elif actual_value == 1:
        value_vec[2] == 1
    else:
        raise Exception('Prediction was not in [1, 0, 2].')
    #value_vec = [0, 0, 1]
    
    #pred[0], pred[1] = pred[1], pred[0]
    
    pred[0], pred[1], pred[2] = pred[2], pred[0], pred[1]   # order: loss, draw, win
    
    #print(pred)
    
    rps = 0
    
    for i in range(0, r-1):    # r-1 becomes r because of the exclusion of range()
        bracket_part = 0
        for j in range(0, i+1):    # same for i and i+1
            bracket_part += pred[j] - value_vec[j]
        
        rps += np.square(bracket_part)
    
    rps *= 1 / (r - 1)
    
    return rps

# Model fit

In [9]:
# Apparently the random states are pretty important. 21 works very well on RPS and test accuracy, 16 only on accuracy.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.222, random_state=20)

## Find best value for *n_estimators*

In [13]:
# We start with n_estimators=50...
xgb_cl = xgb.XGBClassifier(objective='multi:softmax', n_estimators=50, seed=16)
# ...and then validate it to find the lowest loss.
op = xgb_cl.fit(X_train, y_train, early_stopping_rounds=900, eval_metric='mlogloss', eval_set=[(X_test, y_test)])

[0]	validation_0-mlogloss:1.05159


d:\python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.02520
[2]	validation_0-mlogloss:1.00897
[3]	validation_0-mlogloss:0.99943
[4]	validation_0-mlogloss:0.99461
[5]	validation_0-mlogloss:0.98913
[6]	validation_0-mlogloss:0.98577
[7]	validation_0-mlogloss:0.98409
[8]	validation_0-mlogloss:0.98478
[9]	validation_0-mlogloss:0.98430
[10]	validation_0-mlogloss:0.98471
[11]	validation_0-mlogloss:0.98532
[12]	validation_0-mlogloss:0.98546
[13]	validation_0-mlogloss:0.98485
[14]	validation_0-mlogloss:0.98523
[15]	validation_0-mlogloss:0.98631
[16]	validation_0-mlogloss:0.98643
[17]	validation_0-mlogloss:0.98745
[18]	validation_0-mlogloss:0.98898
[19]	validation_0-mlogloss:0.99127
[20]	validation_0-mlogloss:0.99189
[21]	validation_0-mlogloss:0.99234
[22]	validation_0-mlogloss:0.99355
[23]	validation_0-mlogloss:0.99401
[24]	validation_0-mlogloss:0.99533
[25]	validation_0-mlogloss:0.99560
[26]	validation_0-mlogloss:0.99618
[27]	validation_0-mlogloss:0.99740
[28]	validation_0-mlogloss:0.99772
[29]	validation_0-mlogloss:0.

In [24]:
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
n_estimators = [4, 8, 16, 32]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators)

In [26]:
# I tried to do Grid Search here, but the 

grid = GridSearchCV(estimator=xgb_cl, param_grid=param_grid, n_jobs=8, cv=3)
grid_result = grid.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

d:\python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:29:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.475635 using {'learning_rate': 0.2, 'n_estimators': 8}


## Use best parameters to fit the model

In [10]:
# n_estimators=8 had the lowest loss, so we overwrite the previous model.b
xgb_cl = xgb.XGBClassifier(objective='multi:softprob', n_estimators=8, seed=16, learning_rate=0.3)

In [11]:
xgb_cl.fit(X_train, y_train)

d:\python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:11:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=8, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=16, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=16, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# Evaluation

In [12]:
preds = xgb_cl.predict(X_test)
accuracy = float(np.sum(preds == y_test))/y_test.shape[0]

print(f'accuracy: {accuracy}')

accuracy: 0.5264797507788161


In [13]:
# Looking at train set accuracy to get an intuition of how much the model overfits
preds = xgb_cl.predict(X_train)
accuracy = float(np.sum(preds == y_train))/y_train.shape[0]

print(f'accuracy on train set: {accuracy}')

accuracy on train set: 0.613948434258619


In [14]:
# Predictions in form of [0.2, 0.5, 0.3] in the order of draw, home win, away win.
# This is because it gets ordered like 0, 1, 2.
proba_preds = xgb_cl.predict_proba(X_test)

# List of RPS scores for every game in the test set.
# Important to use iloc for y_test, otherwise indices would be wrong
rps_list = [rps(pred, y_test.iloc[i]) for i, pred in enumerate(proba_preds)]

# Average ranked probability score.
np.mean(rps_list)

0.20156425834036565